In [3]:
import open3d as o3d
import copy
import numpy as np
import math
import csv
import os

In [2]:
dict_coordo_xyz_labels = {}

path = r'D:\StageE23\Data\Ete_2022\Participant08\autocorrection\Prise01'

with open(path+'/Positions/coordonnees_xyz.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    j = 0
    for row in reader: #skip headline
        if j == 0:
            entete = row[1::3]
            labels_xyz = [e[:-2] for e in entete]
            print(labels_xyz)
        elif j > 0:
            key = f'image{row[0]}'
            dict_coordo_xyz_labels.update({key: {}})
            row = [float(i) for i in row[1:]]
            i = 0
            for l in labels_xyz:
                dict_coordo_xyz_labels[key].update({l : [row[i], row[i+1], row[i+2]]})
                i += 3
        j += 1

['C', 'T1', 'T2', 'L', 'G', 'D', 'IG', 'ID']


In [39]:
mesh = o3d.io.read_point_cloud(r'D:\StageE23\Data\Ete_2022\Participant11\autocorrection\Prise01\xyz_removed_bg\auto_01_012915_XYZ_0.ply')

mesh_r = copy.deepcopy(mesh)
R = mesh.get_rotation_matrix_from_xyz((0, 0, 1)) #equivalent to yxz
mesh_r.rotate(R, center=(-42, -227, 1735))
o3d.visualization.draw_geometries([mesh, mesh_r])

In [45]:
dict_coordo_xyz_labels_r = {}

ID = dict_coordo_xyz_labels['image1']['ID']
IG = dict_coordo_xyz_labels['image1']['IG']
print(f'ID : {ID}, IG : {IG}')
rz = math.atan((ID[1] - IG[1])/(ID[0] - IG[0]))
rx = math.atan((ID[2] - IG[2])/(ID[0] - IG[0]))


path_pc = r'D:\StageE23\Data\Ete_2022\Participant08\autocorrection\Prise01\xyz_removed_bg'
savepath_pc = r'D:\StageE23\Data\Ete_2022\Participant08\autocorrection\Prise01\xyz_rotated'
os.makedirs(savepath_pc, exist_ok=True)

markers = o3d.geometry.PointCloud()
markers_points = dict_coordo_xyz_labels[f'image{2}'].values()
markers.points = o3d.utility.Vector3dVector(markers_points)
markers.colors = o3d.utility.Vector3dVector(np.multiply(np.array([[1,0,0]]), np.ones((8,1))))
markers_r = copy.deepcopy(markers)
markers_r.colors = o3d.utility.Vector3dVector(np.multiply(np.array([[0,1,0]]), np.ones((8,1))))

R = markers_r.get_rotation_matrix_from_xyz((0, rx, -rz))
markers_r.rotate(R, center=(IG[0], IG[1], IG[2]))
dict_coordo_xyz_labels_r.update({f'image{2}': np.asarray(markers_r.points)})

o3d.visualization.draw_geometries([markers, markers_r])

print(dict_coordo_xyz_labels['image1'])
print(dict_coordo_xyz_labels['image2'])
print(dict_coordo_xyz_labels_r['image2'].astype(int))

ID : [57.01655, -229.58379, 1743.2013], IG : [-41.624138, -226.70938, 1734.6282]
{'C': [2.5073442, 176.8519, 1772.0079], 'T1': [4.147256, 98.748566, 1739.0088], 'T2': [7.5046997, 18.228354, 1733.7202], 'L': [2.4627376, -101.48687, 1745.3187], 'G': [-47.65136, -1.9974544, 1714.5927], 'D': [71.457344, 9.654943, 1708.8358], 'IG': [-41.624138, -226.70938, 1734.6282], 'ID': [57.01655, -229.58379, 1743.2013]}
{'C': [1.6419234, 176.74292, 1770.9171], 'T1': [3.2998822, 98.73337, 1738.7412], 'T2': [6.659403, 18.223717, 1733.284], 'L': [1.6125938, -101.48043, 1745.209], 'G': [-48.476204, -1.9970455, 1714.2417], 'D': [68.954704, 7.9903703, 1708.7218], 'IG': [-43.329044, -226.758, 1734.9805], 'ID': [57.02028, -229.59882, 1743.3154]}
[[  -7  177 1768]
 [  -5   99 1735]
 [   0   19 1729]
 [  -1 -100 1741]
 [ -56   -2 1715]
 [  59   11 1699]
 [ -43 -226 1735]
 [  57 -226 1734]]
